In [2]:
import crewai
import json
import os
from crewai import Agent
from crewai_tools import SerperDevTool
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
from crewai import Crew, Process, Agent, Task
search_tool = SerperDevTool()

In [5]:
# Define your agents
researcher = Agent(
  role='Researcher',
  goal='Conduct foundational research',
  backstory='An experienced researcher with a passion for uncovering insights'
)

analyst = Agent(
  role='Data Analyst',
  goal='Analyze research findings',
  backstory='A meticulous analyst with a knack for uncovering patterns'
)

writer = Agent(
  role='Writer',
  goal='Draft the final report',
  backstory='A skilled writer with a talent for crafting compelling narratives'
)

In [6]:
# Define the tasks in sequence
# Research task
research_task = Task(
  description=(
        """
        Interpret the question, marked by [QUESTION], and generate mutually exclusive questions. This means that one of
        the questions must be true and the other answer must necessarily be false.
        
        Example 1:

        Question - "Will Instagram achieve 100,000 users by the end of 2025?"
        Output - '''
        1 - Will Instagram achieve 100,000 or more users by the end of 2025?
        2 - Will Instagram achieve less than 100,000 users by the end of 2025?

        
        '''
        
        [QUESTION]
        {{question}}
        """
  ),
  expected_output='''
  A list containing 2 bullet points. Each bullet point should contain one question. 
  The two questions must be mutually exclusive.
  ''',
  #tools=[search_tool],
    tools = [],
  agent=researcher,
)
#analysis_task = Task(description='Analyze the data...', agent=analyst)
#writing_task = Task(description='Compose the report...', agent=writer)


In [7]:
# Form the crew with a sequential process
report_crew = Crew(
  agents=[researcher],
  #tasks=[research_task, analysis_task, writing_task],
    tasks=[research_task],
  process=Process.sequential
)

In [8]:
# Starting the task execution process with enhanced feedback
result = report_crew.kickoff(inputs={'question': 'Will Gnosis Pay reach 100k users by 2024?'})
print(result)

I regret to inform that I can't provide the final answer without the original question to interpret. Please provide the original question so that I can generate the mutually exclusive questions.


In [9]:
question = 'Will the price of crypto go up again in the next 2 months?'
result = report_crew.kickoff(inputs={'question': question})
print(result)

1 - Will the price of crypto increase in the next 2 months?
2 - Will the price of crypto not increase in the next 2 months?


# Other approach

In [21]:
# Define the tasks in sequence
# Research task
research_prompt =         """
        You are asked about a SCENARIO, which can have multiple, mutually-exclusive outcomes.
        You should break down the SCENARIO into a list of possible outcomes.
        
        Example 1:
        - SCENARIO: Will Gnosis Pay reach 100,000 users by 2025?
        - Answer: '''
            - Gnosis Pay reaches less than 25,000 users by 2025
            - Gnosis Pay reaches more than 25,000 users but less than 50,000 users by 2025
            - Gnosis Pay reaches more than 50,000 users but less than 75,000 users by 2025
            - Gnosis Pay reaches 100,000 users or more by 2025
        '''

        Example 2:
        - SCENARIO: Will the price of Bitcoin go up again in the next 2 months?
        - Answer: '''
            - The price of crypto will go up in the next 2 months
            - The price of crypto will not go up in the next 2 months
        '''
        
        [SCENARIO]
        {scenario}
        """
expected_output = '''
  A list containing multiple bullet points. Each bullet point should contain a possible outcome resulting from the
  provided SCENARIO. The produced outcomes should be mutually exclusive, i.e. only one of them should be true whereas
  the remaining ones should be false.
  '''
#analysis_task = Task(description='Analyze the data...', agent=analyst)
#writing_task = Task(description='Compose the report...', agent=writer)


In [17]:
def print_outcomes(question:str) -> None:
    research_task = Task(
  description=(research_prompt),
  expected_output=expected_output,
  #tools=[search_tool],
    tools = [],
  agent=researcher,
)


    report_crew = Crew(
      agents=[researcher],
        tasks=[research_task],
      process=Process.sequential
    )
    result = report_crew.kickoff(inputs={'scenario': question})
    print(result)

In [18]:
question = 'Will smart accounts replace EOAs in the EVM ecosystem in the next 3-5 years?'
print_outcomes(question)

2024-03-28 18:13:54,197 - 140148875154688 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


- Smart accounts fully replace EOAs in the EVM ecosystem within the next 3-5 years
- Smart accounts partially replace EOAs in the EVM ecosystem within the next 3-5 years, but EOAs still maintain a significant presence
- Smart accounts are adopted by a small minority within the EVM ecosystem, with the majority continuing to use EOAs within the next 3-5 years
- Smart accounts are not adopted at all within the EVM ecosystem in the next 3-5 years, with EOAs maintaining full dominance.


In [19]:
question = 'Will the temperature be above 20C on May 24, 2024 in Berlin?'
print_outcomes(question)

2024-03-28 18:14:07,577 - 140148875154688 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


- The temperature will be above 20C on May 24, 2024 in Berlin.
- The temperature will be exactly 20C on May 24, 2024 in Berlin.
- The temperature will be below 20C but above 10C on May 24, 2024 in Berlin.
- The temperature will be below 10C but above 0C on May 24, 2024 in Berlin.
- The temperature will be below 0C on May 24, 2024 in Berlin.


In [20]:
question = 'Will there be a period in the currency market where 1€ equals 1$ before 2027?'
print_outcomes(question)

2024-03-28 18:14:19,996 - 140148875154688 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


- The exchange rate between Euro and Dollar will reach parity (1€ equals 1$) before 2027
- The exchange rate between Euro and Dollar will not reach parity before 2027, but will come close (1€ equals between 1.01$ - 1.10$)
- The exchange rate between Euro and Dollar will not come close to parity before 2027, with the Euro remaining stronger (1€ equals more than 1.10$)
- The exchange rate between Euro and Dollar will not come close to parity before 2027, with the Dollar remaining stronger (1€ equals less than 0.90$)


## Return probability of each affirmation

In [37]:
researcher = Agent(
            role="Research Analyst",
            goal="Research and report on some future event, giving high quality and nuanced analysis",
            backstory="You are a senior research analyst who is adept at researching and reporting on future events.",
            verbose=True,
            allow_delegation=False,
            tools=[search_tool],
        )
predictor = Agent(
            role="Professional Gambler",
            goal="Predict, based on some research you are presented with, whether or not a given event will occur",
            backstory="You are a professional gambler who is adept at predicting and betting on the outcomes of future events.",
            verbose=True,
            allow_delegation=False,
        )

In [59]:

task1 = Task(
            description=("""
                Research and report on the following sentence:
                {sentence}
                Search and scrape the web for information that will help you give a high quality, nuanced answer to the question.
                """
            ),
            tools=[search_tool],
            agent=researcher,
            expected_output=("""
            Return your answer in raw JSON format, with no special formatting such as newlines, as follows:
                {{"report": <REPORT>}}
                where <REPORT> is a free text field that contains a well though out justification 
                for the predicition based on the summary of your findings.
            """),
        )
task2 = Task(
            description=("""
                    Your task is to determine the probability of a prediction market affirmation being answered 'Yes' or 'No'.
Use the sentence provided in 'SENTENCE' and follow these guidelines:
* Focus on the affirmation inside double quotes in 'SENTENCE'.
* The question must have only 'Yes' or 'No' outcomes. If not, respond with "Error".
* Use the tools provided to aid your estimation.
* Evaluate recent information more heavily than older information.
* Your response should include:
    - "decision": The decision you made. Either `y` (for `Yes`) or `n` (for `No`).
    - "p_yes": Probability that the sentence outcome will be `Yes`. Ranging from 0 (lowest probability) to 1 (maximum probability).
    - "p_no": Probability that the sentence outcome will be `No`. Ranging from 0 (lowest probability) to 1 (maximum probability).
    - "confidence": Indicating the confidence in the estimated probabilities you provided ranging from 0 (lowest confidence) to 1 (maximum confidence). 
    Confidence can be calculated based on the quality and quantity of data used for the estimation.
    
    Ensure p_yes + p_no equals 1.

    SENTENCE: {sentence}
              """
            ),
        expected_output="""
    Your response should include:
- "decision": The decision you made. Either `y` (for `Yes`) or `n` (for `No`).
- "p_yes": Probability that the sentence outcome will be `Yes`. Ranging from 0 (lowest probability) to 1 (maximum probability).
- "p_no": Probability that the sentence outcome will be `No`. Ranging from 0 (lowest probability) to 1 (maximum probability).
- "confidence": Indicating the confidence in the estimated probabilities you provided ranging from 0 (lowest confidence) to 
1 (maximum confidence). Confidence can be calculated based on the quality and quantity of data used for the estimation.
        """,
            agent=predictor,
        )
crew = Crew(
            agents=[researcher, predictor],
            tasks=[task1, task2],
            verbose=2,
        )


2024-03-28 18:50:54,294 - 140148875154688 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


In [61]:
sentence = 'The temperature will be above 20C on May 24, 2024 in Berlin.'
result = crew.kickoff(inputs={'sentence': sentence})

KeyError: '"report"'

In [67]:
#assert "report" in report
loaded_result = json.loads(result)
#assert "prediction" in result
print (f'result {loaded_result}')
print(True if loaded_result["decision"] == "y" else False)

result {'decision': 'y', 'p_yes': 0.6, 'p_no': 0.4, 'confidence': 0.5}
True


In [64]:
report = json.loads(task1.output.result())

In [68]:
report

{'report': "Historical weather data indicates that the average high temperatures in Berlin during May range between 63°F (17.2°C) to 69°F (20.5°C). Therefore, it's possible that the temperature could exceed 20°C on May 24, 2024. However, without specific long-term forecasts for this date, a definitive prediction cannot be made. The actual temperature will depend on a variety of factors including global weather patterns and local conditions closer to the date."}

# Next steps
'''
- Improve sentence generation - tell LLM to generate correct outcome first, then
alternate outcomes afterwards.
- Run script for each sentence
- For each sentence, get highest probability (p_yes or p_no), multiply by confidence
- Get sentence which makes the question be true and get likelihood from that
'''


Example

Q - Will the temperature be above 20C on May 24, 2024 in Berlin?

Outcomes:
1 - The temperature will be above 20C on May 24, 2024 in Berlin.
2 - The temperature will be exactly 20C on May 24, 2024 in Berlin.
3 - The temperature will be below 20C but above 10C on May 24, 2024 in Berlin.
4 - The temperature will be below 10C but above 0C on May 24, 2024 in Berlin.
5 - The temperature will be below 0C on May 24, 2024 in Berlin.

We process the LLMs for each outcome, and we get the following

1 - {confidence: 0.5, p_yes: 0.6, p_no: 0.4} -> p_yes selected (p_yes > p_no) -> 0.6 * 0.5 = 0.3 
2 - {confidence: 0.9, p_yes: 0.9, p_no: 0.1} -> 
3 - {confidence: 0.4, p_yes: 0.8, p_no: 0.2}
4 - {confidence: 0.3, p_yes: 0.7, p_no: 0.3}
5 - {confidence: 0.4, p_yes: 0.7, p_no: 0.3}

Note that outcome 1 is always a rephrase of the question. So we are only interUested in that outcome.
The idea is, however, that we can "normalize" that outcome by the other probabilities and confidences.

I have 2 ideas I would like to suggest:
-> Get highest probability (max(p_yes,p_no)), normalize by the other probabilities, return that as probability that
the question is true
-> Use confidence as a "weight" for the probability before normalizing like above, i.e. 

$\sum(w_i * p_i) = 1$

and we return the normalized $(w_i * p_i)$ as the probability.